In [ ]:
# Install required packages
!pip install anthropic requests beautifulsoup4 pandas plotly asyncio aiohttp

import os
import json
import time
import asyncio
import aiohttp
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta
from collections import defaultdict, deque
from dataclasses import dataclass
from typing import List, Dict, Optional
import requests
from bs4 import BeautifulSoup
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

# Set up your API key (replace with your actual key)
ANTHROPIC_API_KEY = "your_api_key_here"

In [ ]:
from dataclasses import dataclass # Import the dataclass decorator
from datetime import datetime # Import the datetime class
from typing import Optional, List, Dict # Import Optional from typing module
@dataclass
class Claim:
    text: str
    source: str
    timestamp: datetime
    priority: int  # 1-5, 5 being highest
    category: str  # 'military', 'diplomatic', 'civilian', 'economic'
    location: Optional[str] = None

@dataclass
class VerificationResult:
    claim: Claim
    verdict: str  # 'verified', 'unverified', 'disputed', 'false'
    confidence: float
    indian_sources: List[Dict]
    pakistani_sources: List[Dict]
    neutral_sources: List[Dict]
    evidence: List[str]
    last_updated: datetime

# ... (rest of your code)
@dataclass
class Claim:
    text: str
    source: str
    timestamp: datetime
    priority: int  # 1-5, 5 being highest
    category: str  # 'military', 'diplomatic', 'civilian', 'economic'
    location: Optional[str] = None

@dataclass
class VerificationResult:
    claim: Claim
    verdict: str  # 'verified', 'unverified', 'disputed', 'false'
    confidence: float
    indian_sources: List[Dict]
    pakistani_sources: List[Dict]
    neutral_sources: List[Dict]
    evidence: List[str]
    last_updated: datetime

# Configuration for India-Pakistan conflict monitoring
CONFIG = {
    'indian_sources': [
        'mod.gov.in', 'mea.gov.in', 'pib.gov.in',
        'indianexpress.com', 'thehindu.com', 'indiatimes.com'
    ],
    'pakistani_sources': [
        'ispr.gov.pk', 'mofa.gov.pk', 'pid.gov.pk',
        'dawn.com', 'thenews.com.pk', 'tribune.com.pk'
    ],
    'neutral_sources': [
        'reuters.com', 'apnews.com', 'bbc.com',
        'aljazeera.com', 'un.org', 'icrc.org'
    ],
    'priority_keywords': {
        5: ['casualties', 'killed', 'attack', 'strike', 'nuclear'],
        4: ['ceasefire', 'violation', 'cross-border', 'artillery'],
        3: ['troops', 'movement', 'buildup', 'alert'],
        2: ['diplomatic', 'talks', 'meeting', 'statement'],
        1: ['routine', 'patrol', 'exercise']
    }
}

In [ ]:
class NewsClaimExtractor:
    def __init__(self):
        self.military_terms = [
            'troops', 'soldiers', 'army', 'air force', 'navy',
            'missile', 'artillery', 'drone', 'aircraft', 'helicopter'
        ]
        self.action_terms = [
            'fired', 'shot down', 'attacked', 'violated', 'crossed',
            'deployed', 'mobilized', 'intercepted', 'destroyed'
        ]

    def extract_claims(self, news_text: str, source: str) -> List[Claim]:
        """Extract factual claims from news text"""
        claims = []
        sentences = news_text.split('.')

        for sentence in sentences:
            if any(term in sentence.lower() for term in self.military_terms + self.action_terms):
                priority = self._calculate_priority(sentence)
                category = self._categorize_claim(sentence)

                claim = Claim(
                    text=sentence.strip(),
                    source=source,
                    timestamp=datetime.now(),
                    priority=priority,
                    category=category,
                    location=self._extract_location(sentence)
                )
                claims.append(claim)

        return claims

    def _calculate_priority(self, text: str) -> int:
        text_lower = text.lower()
        for priority, keywords in CONFIG['priority_keywords'].items():
            if any(keyword in text_lower for keyword in keywords):
                return priority
        return 2

    def _categorize_claim(self, text: str) -> str:
        text_lower = text.lower()
        if any(term in text_lower for term in ['military', 'army', 'troops', 'ceasefire']):
            return 'military'
        elif any(term in text_lower for term in ['diplomatic', 'talks', 'minister']):
            return 'diplomatic'
        elif any(term in text_lower for term in ['civilian', 'village', 'population']):
            return 'civilian'
        return 'other'

    def _extract_location(self, text: str) -> Optional[str]:
        locations = ['Kashmir', 'LoC', 'Siachen', 'Rajasthan', 'Punjab', 'Sindh']
        for location in locations:
            if location.lower() in text.lower():
                return location
        return None

In [ ]:
class ConflictFactChecker:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.session = aiohttp.ClientSession()

    async def verify_claim(self, claim: Claim) -> VerificationResult:
        """Verify a claim using multiple sources"""
        # Search across different source categories
        indian_results = await self._search_sources(claim.text, CONFIG['indian_sources'])
        pakistani_results = await self._search_sources(claim.text, CONFIG['pakistani_sources'])
        neutral_results = await self._search_sources(claim.text, CONFIG['neutral_sources'])

        # Analyze results
        verdict, confidence, evidence = self._analyze_results(
            indian_results, pakistani_results, neutral_results
        )

        return VerificationResult(
            claim=claim,
            verdict=verdict,
            confidence=confidence,
            indian_sources=indian_results,
            pakistani_sources=pakistani_results,
            neutral_sources=neutral_results,
            evidence=evidence,
            last_updated=datetime.now()
        )

    async def _search_sources(self, query: str, sources: List[str]) -> List[Dict]:
        """Search specific sources using Claude's web search"""
        results = []

        # This is a simulation - replace with actual API call
        # In real implementation, you would use:
        # response = await anthropic_client.messages.create(
        #     model="claude-3-sonnet-20240229",
        #     messages=[{"role": "user", "content": f"Search for: {query}"}],
        #     tools=[{"type": "web_search"}]
        # )

        # Simulated results for demonstration
        for source in sources[:2]:  # Limit to avoid overwhelming the demo
            results.append({
                'source': source,
                'title': f"Report on {query[:30]}...",
                'snippet': f"According to {source}, {query}",
                'timestamp': datetime.now().isoformat(),
                'url': f"https://{source}/article/12345"
            })

        return results

    def _analyze_results(self, indian: List[Dict], pakistani: List[Dict],
                        neutral: List[Dict]) -> tuple:
        """Analyze results from different sources"""
        # This is a simplified analysis - in production, use NLP
        all_sources = len(indian) + len(pakistani) + len(neutral)

        if all_sources == 0:
            return "unverified", 0.0, ["No sources found"]

        # Simple heuristic for demo
        if len(neutral) > 0:
            confidence = min(0.9, len(neutral) / 3)
            verdict = "verified" if confidence > 0.6 else "disputed"
        else:
            confidence = 0.4
            verdict = "unverified"

        evidence = [
            f"Found {len(indian)} Indian sources",
            f"Found {len(pakistani)} Pakistani sources",
            f"Found {len(neutral)} neutral sources"
        ]

        return verdict, confidence, evidence

    async def close(self):
        await self.session.close()

In [ ]:
class NewsroomDashboard:
    def __init__(self):
        self.active_claims = []
        self.verification_results = []
        self.claim_extractor = NewsClaimExtractor()
        self.fact_checker = None

    def initialize_fact_checker(self, api_key: str):
        self.fact_checker = ConflictFactChecker(api_key)

    async def process_breaking_news(self, news_text: str, source: str):
        """Process breaking news and extract claims"""
        claims = self.claim_extractor.extract_claims(news_text, source)
        self.active_claims.extend(claims)

        # Verify high-priority claims
        for claim in claims:
            if claim.priority >= 3:
                result = await self.fact_checker.verify_claim(claim)
                self.verification_results.append(result)

    def generate_broadcast_card(self, claim_id: int) -> Dict:
        """Generate broadcast-ready fact card"""
        if claim_id >= len(self.verification_results):
            return {"error": "Claim not found"}

        result = self.verification_results[claim_id]

        card = {
            "headline": result.claim.text[:100] + "..." if len(result.claim.text) > 100 else result.claim.text,
            "status": result.verdict.upper(),
            "confidence": f"{result.confidence * 100:.0f}%",
            "last_updated": result.last_updated.strftime("%H:%M IST"),
            "sources": {
                "Indian": len(result.indian_sources),
                "Pakistani": len(result.pakistani_sources),
                "International": len(result.neutral_sources)
            },
            "broadcast_advisory": self._get_broadcast_advisory(result),
            "evidence": result.evidence[:3]  # Top 3 evidence points
        }

        return card

    def _get_broadcast_advisory(self, result: VerificationResult) -> str:
        if result.verdict == "verified" and result.confidence > 0.8:
            return "✓ CLEARED FOR BROADCAST"
        elif result.verdict == "verified" and result.confidence > 0.6:
            return "⚠ BROADCAST WITH CAUTION - Partial verification"
        elif result.verdict == "disputed":
            return "⚡ DISPUTED - Present both perspectives"
        else:
            return "❌ DO NOT BROADCAST - Awaiting verification"

    def display_dashboard(self):
        """Display interactive dashboard"""
        html = """
        <div style="font-family: Arial, sans-serif; background-color: #f5f5f5; padding: 20px; border-radius: 10px;">
            <h2 style="color: #d32f2f; border-bottom: 3px solid #d32f2f; padding-bottom: 10px;">
                🔴 LIVE: India-Pakistan Border Situation Monitor
            </h2>

            <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 20px; margin-top: 20px;">
        """

        for i, result in enumerate(self.verification_results):
            card = self.generate_broadcast_card(i)

            # Color coding based on verification status
            bg_color = {
                "VERIFIED": "#c8e6c9",
                "DISPUTED": "#fff3cd",
                "UNVERIFIED": "#ffcdd2",
                "FALSE": "#ef9a9a"
            }.get(card["status"], "#ffffff")

            html += f"""
                <div style="background-color: {bg_color}; padding: 15px; border-radius: 8px; border: 1px solid #ddd;">
                    <h3 style="margin-top: 0; color: #333;">{card["headline"]}</h3>
                    <div style="display: flex; justify-content: space-between; margin: 10px 0;">
                        <span style="font-weight: bold;">Status: {card["status"]}</span>
                        <span>Confidence: {card["confidence"]}</span>
                    </div>
                    <div style="font-size: 0.9em; color: #666;">
                        Sources: IN:{card["sources"]["Indian"]} |
                        PK:{card["sources"]["Pakistani"]} |
                        INT:{card["sources"]["International"]}
                    </div>
                    <div style="margin-top: 10px; padding: 10px; background-color: rgba(255,255,255,0.7); border-radius: 5px;">
                        <strong>{card["broadcast_advisory"]}</strong>
                    </div>
                    <div style="margin-top: 10px; font-size: 0.85em;">
                        <strong>Evidence:</strong>
                        <ul style="margin: 5px 0; padding-left: 20px;">
                            {"".join([f"<li>{e}</li>" for e in card["evidence"]])}
                        </ul>
                    </div>
                    <div style="text-align: right; font-size: 0.8em; color: #888; margin-top: 10px;">
                        Updated: {card["last_updated"]}
                    </div>
                </div>
            """

        html += """
            </div>

            <div style="margin-top: 30px; padding: 15px; background-color: #e3f2fd; border-radius: 8px;">
                <h3 style="color: #1976d2; margin-top: 0;">📊 Verification Statistics</h3>
                <div style="display: flex; justify-content: space-around; text-align: center;">
                    <div>
                        <div style="font-size: 2em; font-weight: bold; color: #388e3c;">
                            {verified}
                        </div>
                        <div>Verified</div>
                    </div>
                    <div>
                        <div style="font-size: 2em; font-weight: bold; color: #f57c00;">
                            {disputed}
                        </div>
                        <div>Disputed</div>
                    </div>
                    <div>
                        <div style="font-size: 2em; font-weight: bold; color: #d32f2f;">
                            {unverified}
                        </div>
                        <div>Unverified</div>
                    </div>
                </div>
            </div>
        </div>
        """.format(
            verified=sum(1 for r in self.verification_results if r.verdict == "verified"),
            disputed=sum(1 for r in self.verification_results if r.verdict == "disputed"),
            unverified=sum(1 for r in self.verification_results if r.verdict == "unverified")
        )

        display(HTML(html))

In [ ]:
# Initialize the dashboard
ANTHROPIC_API_KEY = "your_api_key_here"
dashboard = NewsroomDashboard()
dashboard.initialize_fact_checker(ANTHROPIC_API_KEY)

# Simulate breaking news from India-Pakistan border (May 2025 scenario)
breaking_news_feeds = [
    {
        "source": "ANI",
        "text": "BREAKING: Indian Army reports 5 Pakistani drones shot down over Kashmir LoC near Poonch sector. Military sources confirm heightened surveillance along border.",
        "time": "09:30 IST"
    },
    {
        "source": "Dawn News",
        "text": "Pakistan military denies Indian claims of drone intrusion. ISPR states all Pakistani drones accounted for. Calls Indian statement propaganda.",
        "time": "09:45 IST"
    },
    {
        "source": "Reuters",
        "text": "UN military observers in Kashmir report unusual activity along LoC. Independent verification of drone incident underway. Both sides on high alert.",
        "time": "10:00 IST"
    },
    {
        "source": "PTI",
        "text": "Defense Ministry confirms ceasefire violation in Poonch sector. Artillery exchanges reported. Local villagers evacuated as precautionary measure.",
        "time": "10:15 IST"
    }
]

# Process the news feeds
async def run_simulation():
    print("🔴 STARTING REAL-TIME FACT-CHECKING SIMULATION")
    print("=" * 50)

    for feed in breaking_news_feeds:
        print(f"\n⚡ NEW BREAKING NEWS from {feed['source']} at {feed['time']}")
        print(f"Content: {feed['text'][:100]}...")

        await dashboard.process_breaking_news(feed['text'], feed['source'])

        # Simulate real-time delay
        await asyncio.sleep(1)

    print("\n✅ FACT-CHECKING COMPLETE - DISPLAYING DASHBOARD")
    print("=" * 50)

    # Display the dashboard
    dashboard.display_dashboard()

    # Close the fact checker
    await dashboard.fact_checker.close()

# Run the simulation
await run_simulation()

🔴 STARTING REAL-TIME FACT-CHECKING SIMULATION

⚡ NEW BREAKING NEWS from ANI at 09:30 IST
Content: BREAKING: Indian Army reports 5 Pakistani drones shot down over Kashmir LoC near Poonch sector. Mili...

⚡ NEW BREAKING NEWS from Dawn News at 09:45 IST
Content: Pakistan military denies Indian claims of drone intrusion. ISPR states all Pakistani drones accounte...

⚡ NEW BREAKING NEWS from Reuters at 10:00 IST
Content: UN military observers in Kashmir report unusual activity along LoC. Independent verification of dron...

⚡ NEW BREAKING NEWS from PTI at 10:15 IST
Content: Defense Ministry confirms ceasefire violation in Poonch sector. Artillery exchanges reported. Local ...

✅ FACT-CHECKING COMPLETE - DISPLAYING DASHBOARD


In [ ]:
def generate_news_ticker(dashboard: NewsroomDashboard):
    """Generate news ticker items based on verification status"""
    ticker_items = []

    for result in dashboard.verification_results:
        if result.verdict == "verified" and result.confidence > 0.7:
            ticker_items.append({
                "text": f"CONFIRMED: {result.claim.text[:80]}...",
                "priority": "high",
                "color": "#2e7d32"
            })
        elif result.verdict == "disputed":
            ticker_items.append({
                "text": f"DISPUTED: {result.claim.text[:80]}... (Multiple conflicting reports)",
                "priority": "medium",
                "color": "#ef6c00"
            })
        else:
            ticker_items.append({
                "text": f"DEVELOPING: {result.claim.text[:80]}... (Awaiting verification)",
                "priority": "low",
                "color": "#616161"
            })

    # Display ticker
    html = """
    <div style="background-color: #000; color: #fff; padding: 10px; border-radius: 5px; margin-top: 20px;">
        <h3 style="color: #ff0000; margin: 0 0 10px 0;">📺 NEWS TICKER</h3>
    """

    for item in ticker_items:
        html += f"""
        <div style="padding: 5px; border-left: 4px solid {item['color']}; margin-bottom: 5px;">
            <span style="color: {item['color']};">●</span> {item['text']}
        </div>
        """

    html += "</div>"
    display(HTML(html))

# Generate news ticker
generate_news_ticker(dashboard)

In [ ]:
class HistoricalContextAnalyzer:
    def __init__(self):
        # Simulated historical database
        self.historical_incidents = [
            {
                "date": "2024-03-15",
                "type": "drone",
                "location": "LoC",
                "verified": False,
                "pattern": "Claimed drone intrusion, later retracted"
            },
            {
                "date": "2024-07-22",
                "type": "ceasefire_violation",
                "location": "Poonch",
                "verified": True,
                "pattern": "Artillery exchange, confirmed by UN observers"
            },
            {
                "date": "2024-11-30",
                "type": "drone",
                "location": "Rajasthan",
                "verified": True,
                "pattern": "Drone shot down, wreckage recovered"
            }
        ]

    def analyze_pattern(self, current_claim: Claim) -> Dict:
        """Analyze if current claim fits historical patterns"""
        claim_type = self._determine_claim_type(current_claim.text)
        similar_incidents = [
            incident for incident in self.historical_incidents
            if incident["type"] == claim_type and incident["location"] in current_claim.text
        ]

        pattern_analysis = {
            "similar_incidents": len(similar_incidents),
            "verification_rate": sum(1 for i in similar_incidents if i["verified"]) / len(similar_incidents) if similar_incidents else 0,
            "common_pattern": self._find_common_pattern(similar_incidents),
            "reliability_score": self._calculate_reliability_score(similar_incidents)
        }

        return pattern_analysis

    def _determine_claim_type(self, text: str) -> str:
        text_lower = text.lower()
        if "drone" in text_lower:
            return "drone"
        elif "ceasefire" in text_lower or "violation" in text_lower:
            return "ceasefire_violation"
        elif "artillery" in text_lower or "shelling" in text_lower:
            return "artillery"
        return "other"

    def _find_common_pattern(self, incidents: List[Dict]) -> str:
        if not incidents:
            return "No historical pattern found"

        patterns = [i["pattern"] for i in incidents]
        # Simple pattern detection
        if len(set(patterns)) == 1:
            return patterns[0]
        return "Mixed patterns observed"

    def _calculate_reliability_score(self, incidents: List[Dict]) -> float:
        if not incidents:
            return 0.5

        verified_count = sum(1 for i in incidents if i["verified"])
        return verified_count / len(incidents)

# Analyze historical context for claims
context_analyzer = HistoricalContextAnalyzer()

print("📚 HISTORICAL CONTEXT ANALYSIS")
print("=" * 40)

for result in dashboard.verification_results:
    pattern = context_analyzer.analyze_pattern(result.claim)

    print(f"\nClaim: {result.claim.text[:80]}...")
    print(f"Similar incidents: {pattern['similar_incidents']}")
    print(f"Historical verification rate: {pattern['verification_rate']:.0%}")
    print(f"Pattern: {pattern['common_pattern']}")
    print(f"Reliability score: {pattern['reliability_score']:.2f}")
    print("-" * 40)

📚 HISTORICAL CONTEXT ANALYSIS

Claim: Artillery exchanges reported...
Similar incidents: 0
Historical verification rate: 0%
Pattern: No historical pattern found
Reliability score: 0.50
----------------------------------------


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_monitoring_dashboard(dashboard: NewsroomDashboard):
    """Create real-time monitoring visualization"""

    # Prepare data for visualization
    priorities = [claim.priority for claim in dashboard.active_claims]
    categories = [claim.category for claim in dashboard.active_claims]
    verdicts = [result.verdict for result in dashboard.verification_results]
    confidences = [result.confidence for result in dashboard.verification_results]

    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Claim Priorities', 'Claim Categories',
                       'Verification Status', 'Confidence Levels'),
        specs=[[{"type": "bar"}, {"type": "pie"}],
               [{"type": "pie"}, {"type": "bar"}]]
    )

    # Priority distribution
    priority_counts = {i: priorities.count(i) for i in range(1, 6)}
    fig.add_trace(
        go.Bar(x=list(priority_counts.keys()), y=list(priority_counts.values()),
               name="Priority", marker_color=['green', 'blue', 'yellow', 'orange', 'red']),
        row=1, col=1
    )

    # Category distribution
    category_counts = {cat: categories.count(cat) for cat in set(categories)}
    fig.add_trace(
        go.Pie(labels=list(category_counts.keys()), values=list(category_counts.values()),
               name="Categories"),
        row=1, col=2
    )

    # Verification status
    verdict_counts = {v: verdicts.count(v) for v in set(verdicts)}
    fig.add_trace(
        go.Pie(labels=list(verdict_counts.keys()), values=list(verdict_counts.values()),
               name="Verdicts", marker_colors=['green', 'red', 'orange', 'gray']),
        row=2, col=1
    )

    # Confidence levels
    fig.add_trace(
        go.Bar(x=[f"Claim {i+1}" for i in range(len(confidences))],
               y=[c*100 for c in confidences],
               name="Confidence %", marker_color='lightblue'),
        row=2, col=2
    )

    # Update layout
    fig.update_layout(
        height=600,
        showlegend=True,
        title_text="Real-time Fact-Checking Monitor - India-Pakistan Border Situation"
    )

    fig.show()

# Create visualization
create_monitoring_dashboard(dashboard)

In [ ]:
def generate_broadcast_report(dashboard: NewsroomDashboard):
    """Generate a formatted report for news producers"""

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M IST")

    report = f"""
FACT-CHECK REPORT: India-Pakistan Border Situation
Generated: {timestamp}
================================================

EXECUTIVE SUMMARY
-----------------
Total Claims Analyzed: {len(dashboard.active_claims)}
Verified: {sum(1 for r in dashboard.verification_results if r.verdict == 'verified')}
Disputed: {sum(1 for r in dashboard.verification_results if r.verdict == 'disputed')}
Unverified: {sum(1 for r in dashboard.verification_results if r.verdict == 'unverified')}

HIGH PRIORITY ALERTS
-------------------
"""

    # Add high priority items
    for result in dashboard.verification_results:
        if result.claim.priority >= 4:
            card = dashboard.generate_broadcast_card(dashboard.verification_results.index(result))
            report += f"""
Claim: {result.claim.text}
Status: {result.verdict.upper()}
Confidence: {result.confidence:.0%}
Advisory: {card['broadcast_advisory']}
Sources: IN:{card['sources']['Indian']} | PK:{card['sources']['Pakistani']} | INT:{card['sources']['International']}
----------------------------------------
"""

    report += """
DETAILED ANALYSIS
-----------------
"""

    for i, result in enumerate(dashboard.verification_results):
        report += f"""
{i+1}. {result.claim.text}
   Source: {result.claim.source}
   Time: {result.claim.timestamp.strftime("%H:%M")}
   Category: {result.claim.category}
   Location: {result.claim.location or 'Unspecified'}
   Verdict: {result.verdict}
   Confidence: {result.confidence:.0%}
   Evidence:
"""
        for evidence in result.evidence:
            report += f"   - {evidence}\n"
        report += "\n"

    report += """
RECOMMENDATIONS FOR BROADCAST
-----------------------------
1. Lead with verified high-priority claims
2. Present disputed claims with clear attribution to sources
3. Avoid broadcasting unverified claims without heavy caveats
4. Monitor for updates on developing situations
5. Cross-reference with UN observer reports when available

END OF REPORT
"""

    # Save to file
    filename = f"fact_check_report_{datetime.now().strftime('%Y%m%d_%H%M')}.txt"
    with open(filename, 'w') as f:
        f.write(report)

    print(f"Report saved as: {filename}")
    return report

# Generate report
report = generate_broadcast_report(dashboard)
print(report[:500] + "...")  # Show first 500 characters

Report saved as: fact_check_report_20250510_1002.txt

FACT-CHECK REPORT: India-Pakistan Border Situation
Generated: 2025-05-10 10:02 IST

EXECUTIVE SUMMARY
-----------------
Total Claims Analyzed: 5
Verified: 1
Disputed: 0
Unverified: 0

HIGH PRIORITY ALERTS
-------------------

Claim: Artillery exchanges reported
Status: VERIFIED
Confidence: 67%
Advisory: ⚠ BROADCAST WITH CAUTION - Partial verification
Sources: IN:2 | PK:2 | INT:2
----------------------------------------

DETAILED ANALYSIS
--------...
